# Modeling II

In [180]:
# imports 
import pandas as pd 
import numpy as np 
from numpy import asarray
#import pymc as pm 

# visualizations
import matplotlib.pyplot as plt 
from matplotlib import style 
%matplotlib inline 
import seaborn as sns 
style.use('fivethirtyeight')

# stats
import statsmodels.api as sm 
import statsmodels.formula.api as smf

# modeling 
from sklearn.linear_model import LinearRegression

# transformers 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

# evaluation metrics 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# run time 
from datetime import datetime as dt

In [35]:
cohort_grads = pd.read_csv('data/cohort_gradrates_22',dtype={'entity_cd':'str'})

In [36]:
cohort_grads.head()

,entity_cd,entity_name,year,subgroup_name,cohort,grad_rate
0,010100010034,Albany High School,2022,Hispanic or Latino,6-Year,75.9
1,010100010034,Albany High School,2022,English Language Learner,4-Year,78.7
2,010100010034,Albany High School,2022,White,4-Year,86.6
3,010100010034,Albany High School,2022,White,5-Year,84.1
4,010100010034,Albany High School,2022,White,6-Year,89.9


In [37]:
cohort_grads.shape

(11982, 6)

## Multilinear Regression

In [76]:
# instead of OHE, to reduce dimensionality, going to use frequency encode
subgroup_norm = cohort_grads.groupby('subgroup_name').size() / len(cohort_grads)

# creating a copy of cohort grad rates 
gradrate_df = cohort_grads.copy()

# freq. encoding subgroup_name 
gradrate_df.subgroup_name = gradrate_df.subgroup_name.apply(lambda x: subgroup_norm[x])
gradrate_df.entity_cd = gradrate_df.entity_cd.astype('int64')
#gradrate_df.cohort = gradrate_df.cohort.astype('category')
gradrate_df.head()

,entity_cd,entity_name,year,subgroup_name,cohort,grad_rate
0,10100010034,Albany High School,2022,0.158822,6-Year,75.9
1,10100010034,Albany High School,2022,0.051661,4-Year,78.7
2,10100010034,Albany High School,2022,0.184026,4-Year,86.6
3,10100010034,Albany High School,2022,0.184026,5-Year,84.1
4,10100010034,Albany High School,2022,0.184026,6-Year,89.9


In [148]:
# cohort_cats =["4-Year", "5-Year","6-Year"]
#ct_preprocess_pipe = ColumnTransformer([("label_enc", LabelEncoder(),['cohort'])])
#lr_pipe = Pipeline([("preprocess", ct_preprocess_pipe),
#                    ("model", LinearRegression())])
#lr_pipe

In [177]:
X = gradrate_df[['entity_cd','subgroup_name','cohort']]
y = gradrate_df.grad_rate

# splitting our data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [188]:
ord_enc = OrdinalEncoder()
ord_enc.fit(asarray(X_train['cohort']).reshape(-1,1))
X_train['cohort_enc'] = ord_enc.transform(asarray(X_train['cohort']).reshape(-1,1))

In [190]:
X_train = X_train[['entity_cd','subgroup_name','cohort_enc']]
X_train

,entity_cd,subgroup_name,cohort_enc
4331,310500011670,0.128943,1.0
8462,353100011047,0.268486,0.0
4324,310500011670,0.268486,2.0
6936,332000011505,0.128943,2.0
10915,580905020001,0.158822,1.0
...,...,...,...
184,22902040001,0.184026,2.0
9505,480101060001,0.158822,2.0
4384,310500860989,0.268486,1.0
8612,353100011605,0.053831,2.0


In [191]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [195]:
# need to ord encode X_test 
ord_enc = OrdinalEncoder()
ord_enc.fit(asarray(X_test['cohort']).reshape(-1,1))
X_test['cohort_enc'] = ord_enc.transform(asarray(X_test['cohort']).reshape(-1,1))

# cleaning up dataframe 
X_test = X_test[['entity_cd','subgroup_name','cohort_enc']]
X_test

KeyError: 'cohort'

In [201]:
preds = lr.predict(X_test)
mean_absolute_error(y_test, preds)

11.052251586141708

## Linear Mixed Effects Modeling 

In [30]:
# splitting data 
X = cohort_grads

X_train, X_test = train_test_split(X, test_size=0.2)

In [32]:
# linear mixed effect with 
# structure = "target variable ~ predictor variable"
cohort_md = smf.mixedlm("grad_rate ~ subgroup_name", X_train, groups= X_train.cohort)
cohort_mdf = cohort_md.fit()

C:\Users\devin\anaconda3\envs\ny-ells-project\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\devin\anaconda3\envs\ny-ells-project\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\Users\devin\anaconda3\envs\ny-ells-project\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [34]:
print(cohort_mdf.summary())

                                    Mixed Linear Model Regression Results
Model:                              MixedLM                  Dependent Variable:                  grad_rate  
No. Observations:                   9585                     Method:                              REML       
No. Groups:                         3                        Scale:                               182.6093   
Min. group size:                    3163                     Log-Likelihood:                      -38554.0173
Max. group size:                    3230                     Converged:                           Yes        
Mean group size:                    3195.0                                                                   
-------------------------------------------------------------------------------------------------------------
                                                                  Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------

## PyMc3 - Bayesian Modeling

Instead of standard OLS regression, where we are trying to understand the influence of, for example, `subgroup_name` on graduation rate, we are interested in knowing whether different schools have different relationships and different baseline `graduation_rates`. 

In [3]:
pymc_df = pd.read_csv('data/2022_NYS_grad-rate.csv')
pymc_df

,entity_cd,entity_name,year,subgroup_name,grad_rate
0,10100010034,Albany High School,2022,Hispanic or Latino,73.7
1,10100010034,Albany High School,2022,White,86.9
2,10100010034,Albany High School,2022,Multiracial,77.4
3,10100010034,Albany High School,2022,English Language Learner,70.4
4,10100010034,Albany High School,2022,Economically Disadvantaged,75.5
...,...,...,...,...,...
4675,671501040002,Warsaw Senior High School,2022,Economically Disadvantaged,88.1
4676,680601060001,Penn Yan Academy,2022,White,91.1
4677,680601060001,Penn Yan Academy,2022,Economically Disadvantaged,88.6
4678,680801040001,Dundee Junior-Senior High School,2022,White,78.6


In [9]:
school_idxs, schools = pd.factorize(pymc_df.entity_name)
coords = {
    "school": schools,
    "obs_id": np.arange(len(school_idxs)),
}

In [10]:
coords

{'school': Index(['Albany High School', 'Green Tech High Charter School',
        'Cohoes High School', 'Colonie Central High School',
        'Albany Leadership Cs-Girls', 'Berne-Knox-Westerlo Junior-Senior Hs',
        'Bethlehem Central Senior High School', 'Ravena-Coeymans-Selkirk Sr Hs',
        'Shaker High School', 'Heatly School',
        ...
        'Barack Obama School For Sj', 'Lakeland High School',
        'Walter Panas High School', 'Yorktown High School',
        'Attica Senior High School', 'Letchworth Senior High School',
        'Perry Junior-Senior High School', 'Warsaw Senior High School',
        'Penn Yan Academy', 'Dundee Junior-Senior High School'],
       dtype='object', length=1240),
 'obs_id': array([   0,    1,    2, ..., 4677, 4678, 4679])}

In [15]:
with pm.Model(coords=coords) as hierarchical_model:
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    
    # Independent parameters for each county
    floor = pm.Data("floor", pymc_df.floor.values, dims="obs_id")
    
    # Hyperpriors for group nodes
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=100)
    sigma_a = pm.HalfNormal("sigma_a", 5.0)
    mu_b = pm.Normal("mu_b", mu=0.0, sigma=100)
    sigma_b = pm.HalfNormal("sigma_b", 5.0)

    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    
    a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="school")
    # effect difference between basement and floor level
    b = pm.Normal("b", mu=mu_b, sigma=sigma_b, dims="school")

    # Model error
    eps = pm.HalfCauchy("eps", 5.0)

    school_est = a[school_idx] + b[school_idx] * data.floor.values

    # Data likelihood
    radon_like = pm.Normal(
        "school_like", mu=radon_est, sigma=eps, observed=data.log_radon, dims="obs_id")

AttributeError: 'DataFrame' object has no attribute 'floor'